# Tarea 1: Piping
### Pablo Correa e Ian Gross

## Librerías

In [1]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
import scipy.constants as cte

from fluids.units import *

g = cte.g*u.m/u.s**2

## Problema 1
En un sistema de refrigeración, el refrigerante R134a fluye a través de un tubo de cobre estándar tipo
L de 3/4 de pulgada. En este sistema, se utiliza un desuperheter para calentar agua, como se muestra en
la Figura P4.3. El R134a ingresa al desuperheter a 200 psia y 225 ºF. El desuperheter extrae suficiente
energía, $Q_{DSH}$, de manera que el R134a ingresa al condensador como un vapor saturado a una presión
levemente inferior. En el condensador, se transfiere energía desde el refrigerante a una tasa de 60000
Btu/h. Esto hace que el R134a salga del condensador como un líquido saturado con una caída de presión
despreciable.

a. Determine la pérdida de carga en el desuperheater.

In [2]:
# Datos
D = (0.75*u.inch).to(u.m)
P1 = (200*u.psi).to(u.Pa) # psia = psi
T1 = (225*u.fahrenheit).to(u.K)
perdida_condensador = (6E4*u.Btu*u.hr**(-1)).to(u.W)

In [3]:
# Propiedades
fluid = "R134a"
epsilon = fld.material_roughness('Copper')

rho = cp.PropsSI('D','Q', 1, 'P', P1.magnitude, fluid)*u.kg/u.m**3

# Calor Qdsh extraido
hi = cp.PropsSI('H','T', T1.magnitude, 'P', P1.magnitude, fluid)*u.J/u.kg
hdsh = cp.PropsSI('H','P', P1.magnitude, 'Q', 1, fluid)*u.J/u.kg
qdsh = hi-hdsh
print(f"Entalpía Inicial: {hi.to(u.kJ/u.kg)}")
print(f"Entalpía de Saturación: {hdsh.to(u.kJ/u.kg)}")
print(f"qdsh = {qdsh.to(u.kJ/u.kg)}")

Entalpía Inicial: 486.5574355995059 kilojoule / kilogram
Entalpía de Saturación: 424.0838375481175 kilojoule / kilogram
qdsh = 62.47359805138834 kilojoule / kilogram


In [4]:
# Pérdida de carga
DP = qdsh*rho
print(DP.to(u.kPa))

4353.343467661183 kilopascal


b. Determine la velocidad (m/s) del R134a al ingresar al condensador.

Considerando la variaciones de estado entre la entrada al desuperheater (1) y su salida (2), se sabe que el flujo másico es constante en el desuperheater por lo tanto se cumple que:
$$m = \rho_1Av_1 = \rho_2Av_2$$
$$v_1 = \frac{\rho_2}{\rho_1}v_2$$

Despreciando el efecto de la conexión entre el desuperheater y el condensador, entonces se puede considerar que la velocidad en la salida del desuperheater es igual a la velocidad en la entrada del condensador.

In [31]:
rho1 = cp.PropsSI('D','T', T1.magnitude, 'P',P1.magnitude, fluid) * u.kg/(u.m)**3
rho2 = cp.PropsSI('D','P', P1.magnitude, 'Q',1, fluid) * u.kg/(u.m)**3

r_rho = rho2/rho1

print(r_rho)

2.4919849641588514 dimensionless


Por Bernoulli y considerando que:
$$\frac{P_1}{\gamma_1} + \frac{V_1^2}{2g} + \cancel{z_1} = \frac{P_2}{\gamma_2} + \frac{V_2^2}{2g} + \cancel{z_2}$$
$$\frac{P_1}{\gamma_1} + \frac{P_2}{\gamma_2} = \frac{V_2^2}{2g} - \frac{V_1^2}{2g}$$
$$\frac{P_1}{\gamma_1} - \frac{P_2}{\gamma_2} = \frac{V_2^2}{2g}(1 - (\frac{\rho_2}{\rho_1})^2)$$

In [27]:
v2 = np.sqrt(P1*(1/rho1 - 1/rho2)*(1-(rho2/rho1)**2)/2)
print(1/rho1 - 1/rho2)
print(1 - (rho2/rho1)**2)

1.0261181147065843 meter ** 3 / kilogram
-5.209989061593792 dimensionless


c. Determine la velocidad (m/s) del R134a al salir del condensador.

d. Investigue sobre correlaciones de pérdida de carga en tuberías con flujo bigfásico. Proponga
una expresión para estimar la pérdida de carga en el condensador. ¿qué información sería necesaria?

e. Cuál es la diferencia entre una tubería tipo L y una tipo M

Las tuberías tipo L tienen un espesor proporcional mayor al de la tuberías tipo M. Esto les permite soportar mayores presiones y, por lo tanto, se utilizan en aplicaciones industriales o de transporte de gas licuado o gas natural. A motivo del mayor requerimiento de material, las tuberías L tienen un costo más elevado. Además, el mayor espesor de la tubería L le da una durabilidad notablemente mayor frente al mismo uso. [1,2]

## Problema 2
Gas Natural fluye con una velocidad promedio de 100 ft/s por una tubería de cobre de 300 m y $1\frac{1}{4}$
inch de diámetro. La temperatura del gas es 20 ◦C y la presión inicial es de 1 psig.

a. Asumiendo que el flujo es incompresible, determine la variación de la presión (pérdida de
carga) en la tubería.

Bernoulli:
$$\frac{P_1}{\gamma} + \cancel{\frac{V_1^2}{2g}} + \cancel{z_1} = \frac{P_2}{\gamma} + \cancel{\frac{V_2^2}{2g}} + \cancel{z_2} + f\frac{L}{D}\frac{V_2^2}{2g}$$
$$\frac{P_1}{\gamma} = \frac{P_2}{\gamma} + f\frac{L}{D}\frac{V^2}{2g}$$
$$P_2 = P_1 - f\frac{L}{D}\frac{\rho V^2}{2}$$

In [5]:
# Datos
V = (100*u.ft*u.s**(-1)).to(u.m*u.s**(-1))
L = 300 * u.m
D = (1.25 * u.inch).to(u.m)
T = (20 * u.celsius).to(u.K)
P1 = ((1+14.7)*u.psi).to(u.Pa) # Está en psiG

Para calcular las propiedades del gas natural, es necesario crear una mezcla. Utilizando los datos de Enel [3], se normalizan los tres componentes más relevantes para obtener:

0.931677% Metano, 0.0660455% Etano, 0.00227743% Propano

In [6]:
# Propiedades
fluid = "Methane[0.931677]&Ethane[0.0660455]&Propane[0.00227743]"
rho1 = cp.PropsSI('D','T', T.magnitude, 'P',P1.magnitude, fluid)*u.kg/u.m**3
mu = cp.PropsSI('V','T', T.magnitude, 'P',P1.magnitude, fluid)*u.Pa*u.s
print(f"rho: {rho1}, mu: {mu}")

rho: 0.758210578030446 kilogram / meter ** 3, mu: 1.089867026972943e-05 pascal * second


In [7]:
# Fricción
Re = (fld.Reynolds(D=D, rho=rho1, V=V, mu=mu)).to('dimensionless')
epsilon = (fld.material_roughness('Copper'))*u.m
f = fld.friction.friction_factor(Re, eD=epsilon/D)
print(f"Re = {Re}, f = {f}")


Re = 67324.79161436268 dimensionless, f = 0.0987876126689345 dimensionless


In [8]:
# Bernoulli
P2 = P1 - (f*rho1*V**2*L)/(2*D)
print(f"P2: {P2}")

P2: -220505.35555028834 pascal


b. Determine el grado de error introducido por el supuesto.

$$\frac{P_1}{g\rho_1} = \frac{P_2}{g\rho_2} + f\frac{L}{D}\frac{V^2}{2g}$$


In [9]:
rho2 = cp.PropsSI('D','T', T.magnitude, 'P',P2.magnitude, fluid)*u.kg/u.m**3
l_izq = (P1/(rho1*g)).to(u.m)
l_der = (P2/(rho2*g)).to(u.m)

print(f"l_izq: {l_izq}, l_der: {l_der}")
print(f"diferencia: {l_izq-l_der}")

ValueError: Brent's method f(b) is NAN for b = 1, other input was a = 0 : PropsSI("D","T",293.15,"P",-220505.3556,"Methane[0.931677]&Ethane[0.0660455]&Propane[0.00227743]")

c. Verifique la validez del supuesto.

d. Haga una evaluación (gráfico) del error vs longitud de la tubería.

e. ¿Cuánto influencia la temperatura en la pérdida de carga? Grafique

## Problema 3
Se utiliza una tubería horizontal para transportar etanol (alcohol etílico) a 20 ◦C a lo largo de una
distancia de 3 km. La tubería está hecha de acero comercial. El caudal requerido de etanol es de 40
L/s. La bomba conectada a esta tubería puede superar una caída de presión de hasta 200 kPa en la
tubería.

a. Especifique el tamaño apropiado (diámetro nominal) de una tubería de tipo Schedule 40 para esta tubería.

In [ ]:
T = (20 * u.celsius).to(u.K)
L = 3E3 * u.m
caudal = 40 * u.liters*u.s**(-1)
caida_P = 200E3 * u.Pa

b. Grafique el caudal máximo que sería entregado por diferentes diametros del schedule
indicado.

## Problema 4
Considere el plano hidráulico de la localidad de Hualañé.

a. Establezca un código computacional, que permita estimar la pérdida de carga máxima en el sistema, cuando todas las casas tienen una llave de
lavamanos abierta.

b. Grafique la pérdida de carga asociada a cada casa si eldiametro de todas las
tuberías es de 1 pulgada.

# Bibliografía

[1] https://woolfplumbing.com.au/blog/the-differences-between-copper-pipes-type-l-and-type-m

[2] https://sodimac.falabella.com/sodimac-cl/page/como-elegir-tuberias-y-fittings-de-cobre

[3] https://www.enel.cl/es/inversionistas/inversionistas-enel-generacion/nuestras-centrales/gasoducto/informacion-general.html